In [11]:
from mpi4py import MPI
import numpy as np
from dolfinx import mesh, fem, io
from dolfinx.fem import FunctionSpace, Function, Constant, dirichletbc
from dolfinx.mesh import create_rectangle, CellType
import ufl
from ufl import dx, inner, grad, sym, Identity, tr
import matplotlib.pyplot as plt

def simple_fatigue_demo():
    """Minimal working example for fatigue analysis"""
    comm = MPI.COMM_WORLD
    
    # Create mesh
    mesh = create_rectangle(comm, [np.array([0, 0]), np.array([1, 0.1])], 
                           [20, 5], CellType.triangle)
    
    # Function spaces - CORRECTED for new FEniCSx
    V = FunctionSpace(mesh, ("Lagrange", 1, (2,)))  # Displacement (vector)
    V_scalar = FunctionSpace(mesh, ("Lagrange", 1))  # Scalar fields
    
    # Material properties
    E = 210.0e9
    nu = 0.3
    lmbda = E * nu / ((1 + nu) * (1 - 2 * nu))
    mu = E / (2 * (1 + nu))
    
    # Functions
    u = Function(V)
    stress = Function(V_scalar)
    
    # Boundary conditions
    def left(x):
        return np.isclose(x[0], 0.0)
    
    left_facets = mesh.locate_entities_boundary(mesh, 1, left)
    left_dofs = fem.locate_dofs_topological(V, 1, left_facets)
    bcs = [dirichletbc(np.zeros(2), left_dofs, V)]
    
    # Define stress and strain
    def epsilon(u):
        return sym(grad(u))
    
    def sigma(u):
        return lmbda * tr(epsilon(u)) * Identity(2) + 2 * mu * epsilon(u)
    
    # Variational problem
    v = ufl.TestFunction(V)
    T = Constant(mesh, (1.0e6, 0.0))  # Load
    
    F = inner(sigma(u), epsilon(v)) * dx - inner(T, v) * dx
    problem = fem.petsc.LinearProblem(F, u, bcs=bcs)
    u_sol = problem.solve()
    
    # Calculate von Mises stress
    sigma_tensor = sigma(u_sol)
    s_xx = sigma_tensor[0, 0]
    s_yy = sigma_tensor[1, 1] 
    s_xy = sigma_tensor[0, 1]
    von_mises = ufl.sqrt(s_xx**2 + s_yy**2 - s_xx*s_yy + 3*s_xy**2)
    
    # Project stress - CORRECTED for new FEniCSx
    L2_error = fem.form(ufl.inner(stress - von_mises, stress - von_mises) * dx)
    error_local = fem.assemble_scalar(L2_error)
    error_global = mesh.comm.allreduce(error_local, op=MPI.SUM)
    
    # For stress projection, we need to solve a projection problem
    w = ufl.TrialFunction(V_scalar)
    v_test = ufl.TestFunction(V_scalar)
    a_proj = ufl.inner(w, v_test) * dx
    L_proj = ufl.inner(von_mises, v_test) * dx
    stress_problem = fem.petsc.LinearProblem(a_proj, L_proj)
    stress_sol = stress_problem.solve()
    stress.x.array[:] = stress_sol.x.array
    
    print(f"Max stress: {np.max(stress.x.array):.2f} Pa")
    
    # Save results
    with io.XDMFFile(comm, "simple_fatigue.xdmf", "w") as xdmf:
        xdmf.write_mesh(mesh)
        u_sol.name = "Displacement"
        xdmf.write_function(u_sol)
        stress.name = "VonMisesStress"
        xdmf.write_function(stress)
    
    print("Simple fatigue analysis completed!")

if __name__ == "__main__":
    simple_fatigue_demo()



TypeError: FunctionSpace.__init__() missing 1 required positional argument: 'cppV'